# Creating Subset of Data with Product Changing in Price, and Normalized Prices

In [1]:
import pandas as pd
import os
from measures import wrangling

In [2]:
# we need normalized prices
raw_data = pd.read_csv('./data/prices_standardized/data_v03b.csv')
# what we want to have overwritten
dataset_old = pd.read_csv('./data/clean/data_v0.1.csv')

In [3]:
 # generating dataset in such a way that pipeline remains unchanged
 df0= dataset_old[dataset_old.columns[:11]] # 'key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory', 'category', 'subCategory', 'stock', 'releaseDate'
 df1= raw_data[raw_data.columns[(len(raw_data.columns)-9-151) : (len(raw_data.columns)-9)]] # 151 days normalized prices
 df2= dataset_old[dataset_old.columns[162:285]] # 123 days sales data
 df3= raw_data[raw_data.columns[(len(raw_data.columns)-9) : (len(raw_data.columns)-8)]] # Boolean whether product changes in price
 df= pd.concat([df0,df1,df2,df3], axis=1, join_axes=[df0.index])
 ###
 # at the beginnging we only want to look at subset of products whose price changes over time
dataset= df.loc[df['changes in price']==True] # this is the subset we'll be working with
del dataset['changes in price']
print(dataset.shape) # should be 1866*246



(1866, 285)


In [4]:
# Drop February price-columns
dataset = wrangling.remove_price_cols(dataset, '2018-02-01', '2018-02-28')
print(dataset.columns)

Index(['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory',
       'category', 'subCategory', 'stock',
       ...
       '2018-01-22_sales', '2018-01-23_sales', '2018-01-24_sales',
       '2018-01-25_sales', '2018-01-26_sales', '2018-01-27_sales',
       '2018-01-28_sales', '2018-01-29_sales', '2018-01-30_sales',
       '2018-01-31_sales'],
      dtype='object', length=285)


In [5]:
# Separate Y labels from X variables
Y_dataset = wrangling.sales_cols(dataset, '2017-10-01', '2018-01-31') 
X_dataset = wrangling.remove_sales_cols(dataset, '2017-10-01', '2018-01-31')

In [6]:
X_dataset = X_dataset.drop('stock', axis=1)
print(X_dataset.columns)

Index(['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory',
       'category', 'subCategory', 'releaseDate',
       ...
       '2018-02-19_prices_normalized', '2018-02-20_prices_normalized',
       '2018-02-21_prices_normalized', '2018-02-22_prices_normalized',
       '2018-02-23_prices_normalized', '2018-02-24_prices_normalized',
       '2018-02-25_prices_normalized', '2018-02-26_prices_normalized',
       '2018-02-27_prices_normalized', '2018-02-28_prices_normalized'],
      dtype='object', length=161)


In [7]:
# Flatten X so that each row is 1 day; we expect 12,824*123 = 1,577,352 rows as result
cols = ['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory', 'category', 'subCategory', 'releaseDate']
X_flat = pd.melt(X_dataset, id_vars=cols, var_name='date', value_name='price')
X_flat = X_flat.sort_values(['key', 'date']).reset_index(drop=True)
print(X_flat.shape)
print(X_flat.tail()) # Quick check of the result

(281766, 12)
              key  pid_x size_x  color brand    rrp  mainCategory  category  \
281761  2286945,5  22869   45,5  weiss  Nike  95.18             1         2   
281762  2286945,5  22869   45,5  weiss  Nike  95.18             1         2   
281763  2286945,5  22869   45,5  weiss  Nike  95.18             1         2   
281764  2286945,5  22869   45,5  weiss  Nike  95.18             1         2   
281765  2286945,5  22869   45,5  weiss  Nike  95.18             1         2   

        subCategory releaseDate                          date     price  
281761          3.0  2017-10-01  2018-02-24_prices_normalized  0.325695  
281762          3.0  2017-10-01  2018-02-25_prices_normalized -4.401747  
281763          3.0  2017-10-01  2018-02-26_prices_normalized -4.401747  
281764          3.0  2017-10-01  2018-02-27_prices_normalized -4.401747  
281765          3.0  2017-10-01  2018-02-28_prices_normalized  0.325695  


In [8]:
# Flatten Y similarly, so that the rows of Y correspond to that of X
Y_flat = pd.melt(Y_dataset, id_vars='key', var_name='date', value_name='sales')
Y_flat = Y_flat.sort_values(['key', 'date']).reset_index(drop=True)
print(Y_flat.shape)
print(Y_flat.tail())

(229518, 3)
              key              date  sales
229513  2286945,5  2018-01-27_sales    0.0
229514  2286945,5  2018-01-28_sales    0.0
229515  2286945,5  2018-01-29_sales    1.0
229516  2286945,5  2018-01-30_sales    0.0
229517  2286945,5  2018-01-31_sales    0.0


# Cleaning data

In [9]:
# Clean 'date' columns to keep only YYYY-MM-DD part
X_flat['date'] = X_flat['date'].str[0:10]
Y_flat['date'] = Y_flat['date'].str[0:10]

In [10]:
X_flat['subCategory'] = X_flat['subCategory'].fillna(0) # Fill blank sub-category with 0
X_flat['size_x'] = X_flat['size_x'].fillna('NA') # Fill blank sizes with 'NA' string
X_flat['price'] = X_flat['price'].fillna(method='bfill') # Fill blank prices with earliest given price

# Feature engineering

In [11]:
print(X_flat.head())

      key  pid_x size_x    color   brand    rrp  mainCategory  category  \
0  10013L  10013      L  schwarz  adidas  69.78             1         7   
1  10013L  10013      L  schwarz  adidas  69.78             1         7   
2  10013L  10013      L  schwarz  adidas  69.78             1         7   
3  10013L  10013      L  schwarz  adidas  69.78             1         7   
4  10013L  10013      L  schwarz  adidas  69.78             1         7   

   subCategory releaseDate        date     price  
0         16.0  2017-10-27  2017-10-01  0.260817  
1         16.0  2017-10-27  2017-10-02  0.260817  
2         16.0  2017-10-27  2017-10-03  0.260817  
3         16.0  2017-10-27  2017-10-04  0.260817  
4         16.0  2017-10-27  2017-10-05  0.260817  


In [12]:
def is_eleventh(row):
    if row['date'][-2:] == '11':
        return 1
    else: 
        return 0

X_flat['is_eleventh'] = X_flat.apply(is_eleventh, axis=1)

In [13]:
def is_crazy_day(row):
    if row['date'] == '2017-11-24': # Black Friday
        return 1
    return 0

X_flat['is_crazy_day'] = X_flat.apply(is_crazy_day, axis=1)

In [14]:
# One-hot encoding days
def is_day(day):
    if row['day_of_week'] == day:
        return 1
    return 0

X_flat['day_of_week'] = pd.to_datetime(X_flat['date']).dt.weekday_name

In [15]:
days = pd.get_dummies(X_flat['day_of_week'], prefix='day')
X_flat = X_flat.join(days)

In [16]:
X_flat['days_since_release'] = (pd.to_datetime(X_flat['date']) - pd.to_datetime(X_flat['releaseDate'])).dt.days

In [17]:
X_flat['price_diff'] = X_flat['price'] - X_flat['rrp']

In [18]:
# One-hot encoding everything else except sub-category (too many of them)
colors = pd.get_dummies(X_flat['color'], prefix='color')
brands = pd.get_dummies(X_flat['brand'], prefix='brand')
main_cats = pd.get_dummies(X_flat['mainCategory'], prefix='maincat')
cats = pd.get_dummies(X_flat['category'], prefix='cat')
sub_cats = pd.get_dummies(X_flat['subCategory'], prefix='subcat')

X_flat = X_flat.join(colors)
X_flat = X_flat.join(brands)
X_flat = X_flat.join(main_cats)
X_flat = X_flat.join(cats)

In [19]:
X_flat.columns

Index(['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory',
       'category', 'subCategory', 'releaseDate', 'date', 'price',
       'is_eleventh', 'is_crazy_day', 'day_of_week', 'day_Friday',
       'day_Monday', 'day_Saturday', 'day_Sunday', 'day_Thursday',
       'day_Tuesday', 'day_Wednesday', 'days_since_release', 'price_diff',
       'color_beige', 'color_blau', 'color_braun', 'color_gelb', 'color_gold',
       'color_grau', 'color_gruen', 'color_khaki', 'color_lila',
       'color_orange', 'color_pink', 'color_rosa', 'color_rot',
       'color_schwarz', 'color_silber', 'color_tuerkis', 'color_weiss',
       'brand_Asics', 'brand_Converse', 'brand_Erima', 'brand_Hummel',
       'brand_Jako', 'brand_Jordan', 'brand_Kempa', 'brand_Lotto',
       'brand_Mizuno', 'brand_New Balance', 'brand_Nike', 'brand_Onitsuka',
       'brand_PUMA', 'brand_Reebok', 'brand_Reusch', 'brand_Sport2000',
       'brand_Stance', 'brand_Uhlsport', 'brand_Under Armour', 'brand_adidas',
      

# Further feature engineering

In [20]:
def marketing_activity(row):
    marketingactivities=['2017-10-11','2017-10-16','2017-11-04','2017-11-11','2017-11-23','2017-11-24',
                     '2017-11-25','2017-11-27','2017-12-03','2017-12-27','2017-12-28','2017-12-31',
                    '2018-01-14','2018-01-22','2018-01-23','2018-01-30','2018-02-06','2018-02-07',
                     '2018-02-20','2018-02-22','2018-02-23']
    if row['date'] in marketingactivities:
        return 1
    else:
        return 0
    
X_flat['marketing_activity'] = X_flat.apply(marketing_activity, axis=1)

In [21]:
# Store dataframes to csv
import os
out_directory = './data/clean'
if not os.path.exists(out_directory): 
    os.makedirs(out_directory)

X_flat.to_csv('{}/nn_X_normalized_prices.csv'.format(out_directory))
Y_flat.to_csv('{}/nn_Y.csv_normalized_prices.csv'.format(out_directory))